<a href="https://colab.research.google.com/github/missakaherath/annotation_schema_conversion_ABSA/blob/main/annotation_schema_conversion_ABSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook includes the code to convert the student feedback dataset from our annotation to the annotation which is needed by ABSA baseline we selected.

In [ ]:
kushan = [['Kushan','1','Final_Dataset_tsv'],['Kushan','2','ratemy_professor_data_from_sorted_list_shuffle_1']]
missaka=[['Missaka','1','Final_Dataset_tsv']]
maduwantha = [['Maduwantha','1','Final_Dataset_tsv'],['Maduwantha','2','ratemy_professor_data_from_sorted_list_shuffle_1'],['Maduwantha','3','additional_100_of_rate_my_proffesor']]
annotators = [kushan,missaka,maduwantha]

aspect_layers = ['Aspect1','Aspect2','Aspect3','Aspect4','Aspect5','Aspect6','Aspect7','Aspect8','Aspect9','Aspect10','Aspect12','Aspect13','Aspect14','Aspect15','Aspect16','Aspect17','Aspect18','Aspect19','Aspect20','Aspect21','Aspect22','Aspect23','Aspect24','Aspect25']
document_level_tag = ['Negative', 'Neutral', 'Positive'];

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install dkpro-cassis > /dev/null
#!pip install nltk
import numpy as np
import re
import math
import json
#import nltk
#nltk.download('punkt')

In [ ]:
!mkdir ./sample_data/processed_data_2

In [ ]:
!pip install allennlp==2.1.0 allennlp-models==2.1.0

from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")

def createDictionary(feedback):  #using allenNLP
  #creating lists to store attributes
  '''token = []
  pos = []
  deprel = []
  head = []
  aspects = []'''
  dependencies = [] 
  output = {}

  results = predictor.predict(
    sentence=feedback
  )

  token = results['words']
  pos = results['pos']
  deprel = results['predicted_dependencies']
  head = results['predicted_heads']

  for i in range(len(token)):
    sub_dependencies = [deprel[i],head[i],i+1]
    dependencies.append(sub_dependencies)
  
  output["sentence"] = feedback
  output["token"] = token
  output["pos"] = pos
  output["deprel"] = deprel
  output["head"] = head
  output["dependencies"] = dependencies
  output["aspects"] = []

  return output

  

     |████████████████████████████████| 585 kB 5.2 MB/s 
     |████████████████████████████████| 407 kB 39.9 MB/s 
     |████████████████████████████████| 131 kB 56.1 MB/s 
     |████████████████████████████████| 124 kB 57.3 MB/s 
     |████████████████████████████████| 1.9 MB 50.3 MB/s 
     |████████████████████████████████| 12.8 MB 22 kB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 259 kB 49.7 MB/s 
     |████████████████████████████████| 1.2 MB 54.0 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 7.8 MB 44.2 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 138 kB 63.3 MB/s 
     |████████████████████████████████| 127 kB 59.2 MB/s 
     |████████████████████████████████| 3.3 MB 50.9 MB/s 
     |████████████████████████████████| 895 kB 35.8 MB/s 
  Created wheel for ov

Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'
downloading: 100%|##########| 73220444/73220444 [00:01<00:00, 42775422.95B/s]


extract data using stanza

In [ ]:
!pip install stanza

     |████████████████████████████████| 342 kB 5.2 MB/s 


In [ ]:
import stanza

In [ ]:
stanza.download('en')

2021-08-15 08:16:45 INFO: Downloading default packages for language: en (English)...


2021-08-15 08:18:05 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
#import stanza
#stanza.download('en')

nlp = stanza.Pipeline('en')

# Processing English text
def processedFeedback(feedback):
  en_doc = nlp(feedback)
  #print(type(en_doc))
  return en_doc

#generating the annotation dict for each feedback
def createDictionary(feedback):   #using Stanza

  #creating lists to store attributes
  token = []
  pos = []
  deprel = []
  head = []
  dependencies = []
  aspects = []
  output = {}

  processed_en_doc = processedFeedback(feedback)
  count = 0
  for i, sent in enumerate(processed_en_doc.sentences):
    #print("[Sentence {}]".format(i+1))
    #print(sent)

    for word in sent.words:
      token.append(word.text)

      count+=1
      dependencies_sub=[]
      dependencies_sub.append(word.deprel)
      dependencies_sub.append(word.head)
      dependencies_sub.append(count)
      dependencies.append(dependencies_sub)
      
      pos.append(word.pos)
      deprel.append(word.deprel)
      head.append(word.head)
      #print(word.text, "\t", word.lemma, "\t", word.pos, "\t", word.head, "\t", word.deprel)

  '''print("\n")
  print("sentence : \t" , feedback, "\n","token : \t", token, "\n", "pos : \t",pos, "\n","deprel : \t", deprel, "\n","head : \t", head, "\n","dependencies : \t", dependencies)
  print("\n")'''
  output["sentence"] = feedback
  output["token"] = token
  output["pos"] = pos
  output["deprel"] = deprel
  output["head"] = head
  output["dependencies"] = dependencies
  output["aspects"] = []

  return output


2021-08-15 08:18:25 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-08-15 08:18:25 INFO: Use device: cpu
2021-08-15 08:18:25 INFO: Loading: tokenize
2021-08-15 08:18:25 INFO: Loading: pos
2021-08-15 08:18:26 INFO: Loading: lemma
2021-08-15 08:18:26 INFO: Loading: depparse
2021-08-15 08:18:26 INFO: Loading: sentiment
2021-08-15 08:18:27 INFO: Loading: ner
2021-08-15 08:18:28 INFO: Done loading processors!


In [ ]:
print(createDictionary("The  bread  is top notch as well."))

{'sentence': 'The  bread  is top notch as well.', 'token': ['The', 'bread', 'is', 'top', 'notch', 'as', 'well', '.'], 'pos': ['DET', 'NOUN', 'AUX', 'ADJ', 'NOUN', 'ADV', 'ADV', 'PUNCT'], 'deprel': ['dep', 'nsubj', 'cop', 'root', 'dep', 'advmod', 'dep', 'punct'], 'head': [2, 4, 4, 0, 4, 7, 4, 4], 'dependencies': [['dep', 2, 1], ['nsubj', 4, 2], ['cop', 4, 3], ['root', 0, 4], ['dep', 4, 5], ['advmod', 7, 6], ['dep', 4, 7], ['punct', 4, 8]], 'aspects': []}


In [ ]:
from cassis import *
import csv
import time
import xml.etree.cElementTree as ET

xmlRoot = ET.Element("sentences")  #root of the xml

sentence_id = 0
with open('/content/drive/Shareddrives/FYP/Annotated Data/Missaka/1/TypeSystem.xml', 'rb') as f:
   typesystem_tmp = load_typesystem(f)

finalOutput = []

for annotator in annotators:
  for path_for_data in annotator:
    with open('/content/drive/Shareddrives/FYP/Annotated Data/'+path_for_data[0]+'/'+path_for_data[1]+'/'+path_for_data[2]+'.xmi', 'rb') as f:
      doc_tmp = load_cas_from_xmi(f, typesystem=typesystem_tmp)

      data = doc_tmp.sofa_string

      #print(data)

      with open('./sample_data/processed_data_2/output_'+path_for_data[0]+'_'+path_for_data[1]+'.tsv', 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        for (_, sentence) in enumerate(doc_tmp.select('webanno.custom.'+"Document_levelopinion")):  #_to get sentences seperately
        
          begin = sentence.begin
          end = sentence.end

          print("\n")
          #print("sentence :",sentence.get_covered_text())

          #sentenceInfo = {aspect_layer : "", }
          #a = "Dr. Carnes was not my favorite. She expected waay too much work for a 1 hr class. She is not very helpful, avoid taking her at all costs."
          tempDict =  createDictionary(sentence.get_covered_text()) #annotate data using stanza 

          xmlSentence = ET.SubElement(xmlRoot, "sentence", id=str(sentence_id))
          xmlText = ET.SubElement(xmlSentence, "text").text = sentence.get_covered_text()

          xmlAspectTerms = ET.SubElement(xmlSentence, "aspectTerms")

          #tempDict =  createDictionary(a)  #testing for a single sentence
          #print(tempDict)

          aspects = []
          
          for (index_aspect_layer, aspect_layer_word) in enumerate(aspect_layers): #iterating through aspect layers
            #print(aspect_layer_word)
            sentence_formated_aspect = ''
            sentence_formated_opinion = ''

            #print("----------new aspect later--------")

            aspectLayerItem = {"term" : "", "polarity": "", "from": 0, "to": 0}

            for (index, aspect_opinion) in enumerate(doc_tmp.select('webanno.custom.'+aspect_layer_word)): #_extracting the aspects
              '''print(aspect_opinion.begin , aspect_opinion.end)
              print(begin , end)'''

              #term = []
              if begin <= aspect_opinion.begin and end >= aspect_opinion.end: #check whether the aspect word is within the sentence
                #if(aspect_opinion.Aspect != None and aspect_opinion.Opinion == None): #aspect word
                  if(aspect_opinion.Aspect != None):
                    term = aspect_opinion.get_covered_text().split()
                    aspectLayerItem["term"] = term

                    #get the index of the aspect word
                    beginIdx = len(sentence.get_covered_text()[:(aspect_opinion.begin-begin)].split())
                    endIdx = len(sentence.get_covered_text()[:(aspect_opinion.end-begin)].split())

                    aspectLayerItem["from"] = beginIdx
                    aspectLayerItem["to"] = endIdx

                    #print(term, aspect_layer_word)
                  if(aspect_opinion.Opinion != None):
                    polarity = aspect_opinion.Opinion

                    if polarity.lower() == "positive":
                      aspectLayerItem["polarity"] = "positive"
                      aspectTerm = ET.SubElement(xmlAspectTerms, "aspectTerm", term="", polarity="positive",  From=str(aspect_opinion.begin), to=str(aspect_opinion.end))
                    elif polarity.lower() == "negative":
                      aspectLayerItem["polarity"] = "negative"
                      aspectTerm = ET.SubElement(xmlAspectTerms, "aspectTerm", term="", polarity="negative",  From=str(aspect_opinion.begin), to=str(aspect_opinion.end))
                    else:
                      aspectLayerItem["polarity"] = "neutral"
                      aspectTerm = ET.SubElement(xmlAspectTerms, "aspectTerm", term="", polarity="neutral",  From=str(aspect_opinion.begin), to=str(aspect_opinion.end))

                    #aspectLayerItem["polarity"] = polarity
                    #print(polarity, aspect_layer_word)

                  if(aspectLayerItem["term"] != "" and aspectLayerItem["polarity"] != ""):
                    aspects.append(aspectLayerItem)
                    aspectLayerItem = {"term" : "", "polarity": "", "from": 0, "to": 0}

                  #print("Aspect opinion (covered text):",aspect_opinion.get_covered_text())
                  #print("begin, end :",(aspect_opinion.begin-begin),(aspect_opinion.end-begin) )
                  #print("begin, end :",(begin),(end) )
                  #print("\n")
              #if(len(aspectLayerItem["term"]) > 0):
          tempDict["aspects"] = aspects
          #print(tempDict)
          finalOutput.append(tempDict)
          time.sleep(1)
          sentence_id+=1
          print("Feedback no :",sentence_id)
          print(tempDict)
          '''break
        break
    break
break'''

#write to xml
tree = ET.ElementTree(xmlRoot)
tree.write("filename.xml")

#print(finalOutput)
trainAmount = math.ceil(sentence_id * 0.76)
trainData = finalOutput[:trainAmount]
testData = finalOutput[trainAmount:]
print("-------------------------------------------------------------------------")
print("Train Data :",len(trainData),"\n", trainData)
print("\n")
print("Test Data :",len(testData),"\n", testData)

#writing data to json files
trainJsonStr = json.dumps(trainData)
trainjsonFile = open("trainDataAllen.json", "w")
trainjsonFile.write(trainJsonStr)
trainjsonFile.close()

testJsonStr = json.dumps(testData)
testjsonFile = open("testDataAllen.json", "w")
testjsonFile.write(testJsonStr)
testjsonFile.close()

print("DONE!!")

Streaming output truncated to the last 5000 lines.


Feedback no : 1722
{'sentence': 'Face a lot of difficulty during lab sessions. Need more help during labs.', 'token': ['Face', 'a', 'lot', 'of', 'difficulty', 'during', 'lab', 'sessions', '.', 'Need', 'more', 'help', 'during', 'labs', '.'], 'pos': ['VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'PUNCT', 'VERB', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'PUNCT'], 'deprel': ['nsubj', 'dep', 'dep', 'prep', 'pcomp', 'prep', 'amod', 'pobj', 'punct', 'nsubj', 'nsubj', 'root', 'prep', 'dep', 'dep'], 'head': [10, 3, 1, 3, 4, 5, 8, 6, 10, 12, 12, 0, 12, 13, 12], 'dependencies': [['nsubj', 10, 1], ['dep', 3, 2], ['dep', 1, 3], ['prep', 3, 4], ['pcomp', 4, 5], ['prep', 5, 6], ['amod', 8, 7], ['pobj', 6, 8], ['punct', 10, 9], ['nsubj', 12, 10], ['nsubj', 12, 11], ['root', 0, 12], ['prep', 12, 13], ['dep', 13, 14], ['dep', 12, 15]], 'aspects': [{'term': ['lab', 'sessions'], 'polarity': 'negative', 'from': 6, 'to': 8}, {'term': ['labs'], 'polari

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
!pwd

/content


In [ ]:
!pwd

/content


In [ ]:
%cd /content

/content


In [ ]:
f = open('testData2.json',)
data = json.load(f)
print("Test size :",len(data))
f.close()

f = open('trainData2.json',)
datatrain = json.load(f)
print("Train size :",len(datatrain))
f.close()


Test size : 712
Train size : 2258


# **Creating the XML files**

In [ ]:
!pip install beautifulsoup4
!pip install lxml

In [ ]:
import xml.etree.cElementTree as ET

root = ET.Element("sentences")
sentence = ET.SubElement(root, "sentence", id="892:1")
text = ET.SubElement(sentence, "text").text = "Boot time is super fast, around anywhere from 35 seconds to 1 minute."
aspectTerms = ET.SubElement(sentence, "aspectTerms")
aspectTerm = ET.SubElement(aspectTerms, "aspectTerm", term="Boot time", polarity="positive",  'from'="0", to="9")

tree = ET.ElementTree(root)
tree.write("filename.xml")

SyntaxError: ignored